# Differential Equations

In [1]:
from IPython.display import display, Markdown
def latexify(x):
    out = '$' + x + '$'
    return out

def lprint(x):
    display(Markdown(latexify(latex(x))))

In [2]:
%run -i 'implicit.py'

In this notebook we will implement Luvunkin? methods, reducing a partial differential equation to a finite dimensional problem

## Encoding differential equations

First we seek a way to encode differential equations in SageMath

For now we will only attempt to solve for functions of one variable,

$$ f : \mathbb{R} \rightarrow \mathbb{R} $$

The simplicity of these functions is that all their higher order derivatives, if defined, are also just functions 

$$ f^{(k)} : \mathbb{R} \rightarrow \mathbb{R} $$

Thus a $k$-th order system of partial differential equations can be thought of as function:


$$ \Phi = \phi \circ P_{k} $$

Where:
    
$$ P_k : C^{\infty}(\mathbb{R}) \rightarrow \mathbb{R}^{k+1} $$

$$ P_k(f)(x) = [x , f(x), f^{1}(x), .... , f^{k}(x) ] $$

And 
$$ \phi : \mathbb{R}^{k+1} \rightarrow \mathbb{R}^n $$

for example if $k = 2$ and then we may attempt to solve:

$$ \Phi(f)(x) = f^{(2)}(x) + \cos(x) f^{1}(x) - 10 $$

This notation allows us to encode partial differential equations as multivariable equations, only omitting equations such as:

$$f^{1}(x - 2) = -f^{2}(x) + f(cos(x)) $$

Which would need an alternative treatment

## Fourier series approximation

Now we recall that functions in $C^{\infty}(\mathbb{R})$ can be approximated by sines and cosine, in a Fourier series

$$f(x) = \frac{a_{0}}{2} + \sum_{n = 1}^{\infty}a_n \cos(nx) + \sum_{n = 1}^{\infty}\sin(nx) $$

If we truncate to say $n = 2$ then we can approximate $C^{\infty}(\mathbb{R})$ by $\mathbb{R}^5$, one dimension for each coefficient.

In [3]:
class PDE():
    def __init__(self, k , phi, phi_var):
        if len(phi_var) != k+2:
            raise(Exception('wrong number of functions given, should be a list {} long'.format(k)))
        self.k = k # order of the PDE
        self.phi = phi # list of $k + 1$ functions
        self.phi_var = phi_var # should be a dictionary
        
    
    def __str__(self):
        # reformat phi so shows as a pde
        out = latex(self.phi)
        for var in self.phi_var.keys():
            if var == 'x':
                continue
            out = out.replace(latex(self.phi_var[var]), var)
            
        return out
    
    def _latex_(self):
        return str(self)               
            
        
        
    def fourier_approx_raw(self, x, anbn, n = 2, p = 2*pi):
        # n is the order of the Fourier approximation
        # x should be a SageMath variable
        if len(anbn) != 2*n + 1:
            raise(Exception('Should be {} coefficients'.format(2*n+1)))
        
        # first generate the fourier approx f(x)
        
        f(x) = anbn[0]/2 
        for i in range(1,n+1):
            f(x) = f(x) + anbn[i] * cos(i*x * (2*pi / p)) + anbn[n + i] * sin(i*x * (2*pi / p))
        
        temp(x) = x
        fr_vars = [temp, f]
        if self.k > 0:
            for i in range(1, self.k +1):
                f = f.diff()[0] # returns a matrix of one, so take first elt
                fr_vars.append(f)
        out(x) = phi(*fr_vars)
        
        return out
    
    def fourier_approx(self, x, anbn, n = 2, p = 2*pi):
        raw = self.fourier_approx_raw(x, anbn, n, p)
        out = [0]*(2*n+1)
        
        # first get a0
        a0 = integral(raw, (x,0,p))
        
        out[0] = a0
        for i in range(1, n+1):
            ai = integral(raw * cos(i*x * (2*pi / p)), (x,0,p))
            bi = integral(raw * sin(i*x * (2*pi / p)), (x,0,p))
            out[i] = ai
            out[n + i] = bi
        
        return out
        
        
    # TODO overide display method, so can view the PDE

In [4]:
var('x a0 a1 a2 b1 b2 x0 x1')
var('l', latex_name=r'\lambda')

l

In [5]:
phi(x, x0, x1) = x0*cos(x) + l*x1 - x0^2  # solving f'(x) - f(x)^2 + cos(x) = 0

In [6]:
pde = PDE(1, phi, phi_var = {'x' : x, 'f(x)' : x0, 'f^{1}(x)' : x1} )

In [7]:
lprint(pde)

$ \left( x, f(x), f^{1}(x) \right) \ {\mapsto} \ -f(x)^{2} + {\lambda} f^{1}(x) + f(x) \cos\left(x\right) $

See that the zero function is a solution of this system

In [8]:
f = pde.fourier_approx_raw(x, [a0, a1, a2, b1, b2])

In [9]:
lprint(f)

$ x \ {\mapsto}\ -\frac{1}{4} \, {\left(2 \, a_{2} \cos\left(2 \, x\right) + 2 \, a_{1} \cos\left(x\right) + 2 \, b_{2} \sin\left(2 \, x\right) + 2 \, b_{1} \sin\left(x\right) + a_{0}\right)}^{2} + {\left(2 \, b_{2} \cos\left(2 \, x\right) + b_{1} \cos\left(x\right) - 2 \, a_{2} \sin\left(2 \, x\right) - a_{1} \sin\left(x\right)\right)} {\lambda} + \frac{1}{2} \, {\left(2 \, a_{2} \cos\left(2 \, x\right) + 2 \, a_{1} \cos\left(x\right) + 2 \, b_{2} \sin\left(2 \, x\right) + 2 \, b_{1} \sin\left(x\right) + a_{0}\right)} \cos\left(x\right) $

Here we have replaced $f(x)$ with a Fourier series, and then computed derivatives from those to act as new inputs to phi, now we would like to return this to a standard fourier series

In [10]:
flist = pde.fourier_approx(x, [a0, a1, a2, b1, b2])

In [11]:
for i in flist:
    lprint(i)

$ -\frac{1}{2} \, \pi a_{0}^{2} - \pi a_{1}^{2} - \pi a_{2}^{2} - \pi b_{1}^{2} - \pi b_{2}^{2} + \pi a_{1} $

$ -\pi a_{0} a_{1} + \frac{1}{2} \, \pi a_{0} + \frac{1}{2} \, {\left(\pi - 2 \, \pi a_{1}\right)} a_{2} - \frac{1}{30} \, {\left(30 \, \pi b_{1} - 20 \, a_{0} - 30 \, a_{1} - 8 \, a_{2} + 15\right)} b_{2} - \frac{1}{30} \, {\left(20 \, a_{0} + 30 \, a_{1} + 8 \, a_{2} - 15\right)} b_{2} + \frac{1}{6} \, {\left(6 \, \pi b_{1} + 3 \, a_{1} + 8 \, a_{2}\right)} {\lambda} - \frac{1}{6} \, {\left(3 \, a_{1} + 8 \, a_{2}\right)} {\lambda} $

$ -\frac{1}{2} \, \pi a_{1}^{2} - \pi a_{0} a_{2} + \frac{1}{2} \, \pi b_{1}^{2} + \frac{1}{2} \, \pi a_{1} + \frac{1}{6} \, {\left(12 \, \pi b_{2} - 2 \, a_{1} + 3 \, a_{2}\right)} {\lambda} + \frac{1}{6} \, {\left(2 \, a_{1} - 3 \, a_{2}\right)} {\lambda} $

$ -{\left(\pi a_{0} - \pi a_{2}\right)} b_{1} + \frac{1}{2} \, {\left(\pi - 2 \, \pi a_{1}\right)} b_{2} - \frac{1}{6} \, {\left(6 \, \pi a_{1} + 3 \, b_{1} - 4 \, b_{2}\right)} {\lambda} + \frac{1}{6} \, {\left(3 \, b_{1} - 4 \, b_{2}\right)} {\lambda} $

$ \frac{1}{2} \, {\left(\pi - 2 \, \pi a_{1}\right)} b_{1} - \frac{1}{15} \, {\left(15 \, \pi a_{0} - 16 \, b_{1}\right)} b_{2} - \frac{16}{15} \, b_{1} b_{2} - \frac{1}{6} \, {\left(12 \, \pi a_{2} + 4 \, b_{1} + 3 \, b_{2}\right)} {\lambda} + \frac{1}{6} \, {\left(4 \, b_{1} + 3 \, b_{2}\right)} {\lambda} $

This reduces our problem from an infinite dimensional one to trying to solve:
$\tilde{f} : \mathbb{R}^5 \rightarrow \mathbb{R}^5$

## Dimension reduction

Lets define a system:

In [12]:
f1(a0, a1, a2, b1, b2, l) = flist[0]
f2(a0, a1, a2, b1, b2, l) = flist[1]
f3(a0, a1, a2, b1, b2, l) = flist[2]
f4(a0, a1, a2, b1, b2, l) = flist[3]
f5(a0, a1, a2, b1, b2, l) = flist[4]
funcs = [f1,f2,f3,f4,f5]

In [13]:
lprint(f2)

$ \left( a_{0}, a_{1}, a_{2}, b_{1}, b_{2}, {\lambda} \right) \ {\mapsto} \ -\pi a_{0} a_{1} + \frac{1}{2} \, \pi a_{0} + \frac{1}{2} \, {\left(\pi - 2 \, \pi a_{1}\right)} a_{2} - \frac{1}{30} \, {\left(30 \, \pi b_{1} - 20 \, a_{0} - 30 \, a_{1} - 8 \, a_{2} + 15\right)} b_{2} - \frac{1}{30} \, {\left(20 \, a_{0} + 30 \, a_{1} + 8 \, a_{2} - 15\right)} b_{2} + \frac{1}{6} \, {\left(6 \, \pi b_{1} + 3 \, a_{1} + 8 \, a_{2}\right)} {\lambda} - \frac{1}{6} \, {\left(3 \, a_{1} + 8 \, a_{2}\right)} {\lambda} $

Lets check that zero is still a solution:

In [14]:
[func(0,0,0,0,0,l) for func in funcs]

[0, 0, 0, 0, 0]

So now we compute the Jacobian as usual:

In [15]:
J = jacobian(funcs, (a0, a1, a2, b1, b2))(a0=0, a1=0, a2=0, b1=0, b2=0, l=0 )
lprint(J)

$ \left(\begin{array}{rrrrr}
0 & \pi & 0 & 0 & 0 \\
\frac{1}{2} \, \pi & 0 & \frac{1}{2} \, \pi & 0 & 0 \\
0 & \frac{1}{2} \, \pi & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & \frac{1}{2} \, \pi \\
0 & 0 & 0 & \frac{1}{2} \, \pi & 0
\end{array}\right) $

In [16]:
K = J.right_kernel().basis()
lprint(K)

$ \left[\left(1,\,0,\,-1,\,0,\,0\right)\right] $

We see that the Jacobian has a one dimensional kernel at 0, indicating a potential new strand of solutions:

In [17]:
I = J.image().basis()
lprint(I)

$ \left[\left(1,\,0,\,1,\,0,\,0\right), \left(0,\,1,\,0,\,0,\,0\right), \left(0,\,0,\,0,\,1,\,0\right), \left(0,\,0,\,0,\,0,\,1\right)\right] $

And as expected a 4 dimensional solution set

In [18]:
Kperp = matrix(K).right_kernel().basis()
lprint(Kperp)

$ \left[\left(1,\,0,\,1,\,0,\,0\right), \left(0,\,1,\,0,\,0,\,0\right), \left(0,\,0,\,0,\,1,\,0\right), \left(0,\,0,\,0,\,0,\,1\right)\right] $

In [19]:
Iperp = matrix(I).right_kernel().basis()
lprint(Iperp)

$ \left[\left(1,\,0,\,-1,\,0,\,0\right)\right] $

### Getting the A,B matrices

In [20]:
A = matrix(K+Kperp).inverse()
lprint(A) # inverse so is change of basis

$ \left(\begin{array}{rrrrr}
\frac{1}{2} & \frac{1}{2} & 0 & 0 & 0 \\
0 & 0 & 1 & 0 & 0 \\
-\frac{1}{2} & \frac{1}{2} & 0 & 0 & 0 \\
0 & 0 & 0 & 1 & 0 \\
0 & 0 & 0 & 0 & 1
\end{array}\right) $

In [21]:
B = matrix(I + Iperp).inverse()
lprint(B)

$ \left(\begin{array}{rrrrr}
\frac{1}{2} & 0 & 0 & 0 & \frac{1}{2} \\
0 & 1 & 0 & 0 & 0 \\
\frac{1}{2} & 0 & 0 & 0 & -\frac{1}{2} \\
0 & 0 & 1 & 0 & 0 \\
0 & 0 & 0 & 1 & 0
\end{array}\right) $

### The new f

In [22]:
f1A(a0, a1, a2, b1, b2, l) = f1(*list(A*vector([a0, a1, a2, b1, b2])), l)
f2A(a0, a1, a2, b1, b2, l) = f2(*list(A*vector([a0, a1, a2, b1, b2])), l)
f3A(a0, a1, a2, b1, b2, l) = f3(*list(A*vector([a0, a1, a2, b1, b2])), l)
f4A(a0, a1, a2, b1, b2, l) = f4(*list(A*vector([a0, a1, a2, b1, b2])), l)
f5A(a0, a1, a2, b1, b2, l) = f5(*list(A*vector([a0, a1, a2, b1, b2])), l)

In [23]:
[ft1,ft2,ft3,ft4,ft5] = B.inverse()*vector([f1A,f2A,f3A,f4A,f5A])
funcs = [ft1,ft2,ft3,ft4,ft5]

In [24]:
lprint(ft2)

$ \left( a_{0}, a_{1}, a_{2}, b_{1}, b_{2}, {\lambda} \right) \ {\mapsto} \ -\frac{1}{2} \, \pi {\left(a_{0} + a_{1}\right)} a_{2} + \frac{1}{4} \, \pi {\left(a_{0} + a_{1}\right)} - \frac{1}{4} \, {\left(\pi - 2 \, \pi a_{2}\right)} {\left(a_{0} - a_{1}\right)} - \frac{1}{30} \, {\left(30 \, \pi b_{1} - 6 \, a_{0} - 14 \, a_{1} - 30 \, a_{2} + 15\right)} b_{2} - \frac{1}{30} \, {\left(6 \, a_{0} + 14 \, a_{1} + 30 \, a_{2} - 15\right)} b_{2} + \frac{1}{6} \, {\left(6 \, \pi b_{1} - 4 \, a_{0} + 4 \, a_{1} + 3 \, a_{2}\right)} {\lambda} + \frac{1}{6} \, {\left(4 \, a_{0} - 4 \, a_{1} - 3 \, a_{2}\right)} {\lambda} $

In [25]:
Jt = jacobian(funcs, (a0, a1, a2, b1, b2))(a0=0, a1=0, a2=0, b1=0, b2=0, l=0)
lprint(Jt)

$ \left(\begin{array}{rrrrr}
0 & 0 & \frac{3}{2} \, \pi & 0 & 0 \\
0 & \frac{1}{2} \, \pi & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & \frac{1}{2} \, \pi \\
0 & 0 & 0 & \frac{1}{2} \, \pi & 0 \\
0 & 0 & \frac{1}{2} \, \pi & 0 & 0
\end{array}\right) $

In [26]:
Kt = Jt.right_kernel().basis()
lprint(Kt)

$ \left[\left(1,\,0,\,0,\,0,\,0\right)\right] $

In [27]:
It = Jt.image().basis()
lprint(It)

$ \left[\left(0,\,1,\,0,\,0,\,0\right), \left(0,\,0,\,1,\,0,\,0\right), \left(0,\,0,\,0,\,1,\,0\right), \left(0,\,0,\,0,\,0,\,1\right)\right] $

So we see all is working as expected, we can run the implicit function code on the last four functions

In [28]:
funcs = [ft2,ft3,ft4,ft5]
position = {'a0' : 0, 'a1' : 0, 'a2' : 0, 'b1' : 0, 'b2' : 0, 'l' : 0} # drawback that have to write this way
var_dict = {'y1' : a1, 'y2' : a2, 'y3' : b1, 'y4' : b2, 'x1' : a0, 'x2' : l}
x_var_keys = ['x1', 'x2']
x_dim = 2
y_dim = 4

In [29]:
t_dict = TensorDict(funcs, position, var_dict, x_dim, y_dim)

computing Taylor approximaton to 3 order for speedup


In [30]:
out = get_hkx_polynomial(funcs, 3, x_dim, y_dim, var_dict, x_var_keys, t_dict, position)

100%|██████████| 1/1 [00:00<00:00,  2.46it/s]


[1/2*pi      0      0      0]
[     0      0      0 1/2*pi]
[     0      0 1/2*pi      0]
[     0 1/2*pi      0      0]


100%|██████████| 4/4 [00:09<00:00,  2.28s/it]


[1/2*pi      0      0      0]
[     0      0      0 1/2*pi]
[     0      0 1/2*pi      0]
[     0 1/2*pi      0      0]


100%|██████████| 14/14 [00:30<00:00,  2.16s/it]


[1/2*pi      0      0      0]
[     0      0      0 1/2*pi]
[     0      0 1/2*pi      0]
[     0 1/2*pi      0      0]


In [31]:
lprint(out)

$ \left[8 \, a_{0} {\lambda}^{2}, \frac{5}{4} \, a_{0}^{2}, -2 \, a_{0} {\lambda}, -3 \, a_{0}^{2} {\lambda}\right] $

This is the parametrisation of the zero set of the 2nd-5th compoments of $\tilde{F} = B^{-1} \circ F \circ A$, lets substitute into the first equation, we'll only require the first two terms in its taylor expansion

In [32]:
ft1_taylor = ft1.taylor(*[(key,value) for key,value in position.items()],3)
# unpacking the position we take the expansion at
lprint(ft1_taylor.polynomial(SR))

$ \left(-\frac{1}{8} \, \pi\right) a_{0}^{2} + \left(\frac{1}{4} \, \pi\right) a_{0} a_{1} + \left(-\frac{5}{8} \, \pi\right) a_{1}^{2} + \left(-\frac{3}{2} \, \pi\right) a_{2}^{2} + \left(-\frac{1}{2} \, \pi\right) b_{1}^{2} + \left(-\pi\right) b_{2}^{2} + \left(2 \, \pi\right) b_{2} l + \left(\frac{3}{2} \, \pi\right) a_{2} $

In [33]:
ft1_sub = ft1_taylor(a1 = out[0], a2 = out[1], b1 = out[2], b2 = out[3])
poly = ft1_sub.polynomial(SR)
lprint(poly)

$ \left(-9 \, \pi\right) a_{0}^{4} l^{2} + \left(-40 \, \pi\right) a_{0}^{2} l^{4} + \left(-\frac{75}{32} \, \pi\right) a_{0}^{4} + \left(-6 \, \pi\right) a_{0}^{2} l^{2} + \left(\frac{7}{4} \, \pi\right) a_{0}^{2} $

In [34]:
lprint(solve(symbolic_expression(poly), a0, l))

$ \left[\left[a_{0} = 0, {\lambda} = r_{1}\right], \left[a_{0} = r_{2}, {\lambda} = \frac{1}{4} \, \sqrt{-\frac{9}{5} \, r_{2}^{2} - \frac{1}{5} \, \sqrt{81 \, r_{2}^{4} - 267 \, r_{2}^{2} + 316} - \frac{6}{5}}\right], \left[a_{0} = r_{3}, {\lambda} = -\frac{1}{4} \, \sqrt{-\frac{9}{5} \, r_{3}^{2} - \frac{1}{5} \, \sqrt{81 \, r_{3}^{4} - 267 \, r_{3}^{2} + 316} - \frac{6}{5}}\right], \left[a_{0} = r_{4}, {\lambda} = \frac{1}{4} \, \sqrt{-\frac{9}{5} \, r_{4}^{2} + \frac{1}{5} \, \sqrt{81 \, r_{4}^{4} - 267 \, r_{4}^{2} + 316} - \frac{6}{5}}\right], \left[a_{0} = r_{5}, {\lambda} = -\frac{1}{4} \, \sqrt{-\frac{9}{5} \, r_{5}^{2} + \frac{1}{5} \, \sqrt{81 \, r_{5}^{4} - 267 \, r_{5}^{2} + 316} - \frac{6}{5}}\right]\right] $

Here we see that a non-trivial solutions seem possible, so there is potential for bifurcations. 

## Elastic rod

we wish to encode the problem:
    
$f^{2}(x) + \lambda sin (f(x)) = 0 $

$f^{1}(0) = f^{1}(2 \pi) = 0 $

In [35]:
var('x a0 a1 a2 b1 b2 x0 x1 x2')
var('l', latex_name=r'\lambda')

phi(x, x0, x1, x2) = x2 + l * sin(x0) 
lprint(phi)

$ \left( x, x_{0}, x_{1}, x_{2} \right) \ {\mapsto} \ {\lambda} \sin\left(x_{0}\right) + x_{2} $

In [36]:
pde = PDE(2, phi, phi_var = {'x' : x, 'f(x)' : x0, 'f^{1}(x)' : x1, 'f^{2}(x)' : x2} )
lprint(pde)

$ \left( x, f(x), f^{1}(x), f^{2}(x) \right) \ {\mapsto} \ {\lambda} \sin\left(f(x)\right) + f^{2}(x) $

In [37]:
f = pde.fourier_approx_raw(x, [a0, a1, a2, b1, b2])
lprint(f)

$ x \ {\mapsto}\ -4 \, a_{2} \cos\left(2 \, x\right) - a_{1} \cos\left(x\right) + {\lambda} \sin\left(a_{2} \cos\left(2 \, x\right) + a_{1} \cos\left(x\right) + b_{2} \sin\left(2 \, x\right) + b_{1} \sin\left(x\right) + \frac{1}{2} \, a_{0}\right) - 4 \, b_{2} \sin\left(2 \, x\right) - b_{1} \sin\left(x\right) $

In [38]:
flist = pde.fourier_approx(x, [a0, a1, a2, b1, b2])
for i in flist:
    lprint(i)

Interrupting Giac...


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.


KeyboardInterrupt



Python chokes here, perhaps a new approach

In [ ]:
phi(l, x0, x2) = l*sin(x0) +x2

In [ ]:
J = jacobian([phi],(l, x0, x2))(l=0,x0=0,x2=0)
lprint(J)

In [ ]:
K = J.right_kernel().basis()
lprint(K)

In [ ]:
I = J.image().basis()
lprint(I)

In [ ]:
funcs = [phi]
position = {'x0' : 0, 'x2' : 0, 'l' : 0} # drawback that have to write this way
var_dict = {'y1' : x2, 'x1' : x0, 'x2' : l}
x_var_keys = ['x1','x2']
x_dim = 2
y_dim = 1

In [ ]:
t_dict = TensorDict(funcs, position, var_dict, x_dim, y_dim)

In [ ]:
out = get_hkx_polynomial(funcs, 5, x_dim, y_dim, var_dict, x_var_keys, t_dict, position)

In [ ]:
lprint(out)

In [ ]:
lprint(solve(out, l, x0))

In [ ]:
lprint(phi)

In [ ]:
phi(l = 0.1, x0 = 0.1, x2 = out[0](l = 0.1, x0 = 0.1))